In [1]:
"""
initiate the neural network, not the best design, but it works
"""
from ClassFiles.networks_testing import ConvNet
NN = ConvNet(1, 128, 128)
NN.to("cuda")

ConvNet(
  (poolavg): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (poolmax): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (conv2): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), padding_mode=reflect)
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), padding_mode=reflect)
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), padding_mode=reflect)
  (conv5): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), padding_mode=reflect)
  (conv6): Conv2d(256, 512, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), padding_mode=reflect)
  (conv7): Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1))
)

In [2]:
"""
training, saves to local_testing_net2, epochs = 15, batchsie = 50,
added an option to shuffle to training data or not
(not shuffling means it is trained on pais of corresponding chanvese, groundtruth images)

in algorithm1_testing it changes each segmentation to a binary image, subtracts -0.5 to make black 0.5, white -0.5, and randomly flips the colours
"""
from algorithm1_testing import train
NN = train(NN, 'local_testing_net3', 15, batch_size = 50 , mu = 1, shuffle = True, flip = False)

untrained performance 0.7812652587890625 -126.23016357421875 -136.70608520507812


RuntimeError: CUDA out of memory. Tried to allocate 100.00 MiB (GPU 0; 3.00 GiB total capacity; 1.81 GiB already allocated; 62.05 MiB free; 1.83 GiB reserved in total by PyTorch)

In [2]:
import torch
from ClassFiles.DataLoader import get_generated_dataloader
from algorithm2_old import reconstruct, shitty_reconstruct, minimum

eval_groundtruth_loader = get_generated_dataloader('eval', 'clean', batch_size = 100, shuffle = False)
eval_chanvese_loader = get_generated_dataloader('eval', 'chan-vese', batch_size = 100, shuffle = False)
eval_noisy_loader = get_generated_dataloader('eval', 'dirty', batch_size = 100, shuffle = False)

eval_groundtruth_batch = iter(eval_groundtruth_loader).next()[0].to("cuda") - 0.5
eval_chanvese_batch = iter(eval_chanvese_loader).next()[0].to("cuda")
eval_chanvese_batch = (eval_chanvese_batch > 0.5).float() - 0.5
eval_noisy_batch = iter(eval_noisy_loader).next()[0].to("cuda") - 0.5

NN.to("cuda")

#reconstructed_batch, quality, steps = minimum(eval_chanvese_batch, eval_noisy_batch, eval_groundtruth_batch, NN, 1, 0.001)
#print(steps)

#reconstructed_batch, _ = reconstruct(eval_chanvese_batch, eval_noisy_batch, NN, 0.001, 0.1, 400) #c1 = torch.zeros([1], device = "cuda"), c2 = torch.ones([1], device = "cuda"))

#reconstructed_batch, _ = reconstruct(eval_noisy_batch, eval_noisy_batch, NN, 0, 16000, 400) #c1 = torch.zeros([1], device = "cuda"), c2 = torch.ones([1], device = "cuda"))

reconstructed_batch, _ = shitty_reconstruct(eval_chanvese_batch, eval_noisy_batch, NN, 0.01, 50)

print(NN(reconstructed_batch).mean().item(), NN(eval_groundtruth_batch).mean().item(), NN(eval_chanvese_batch).mean().item()) 

reconstructed_batch_numpy = reconstructed_batch.to("cpu").numpy()
reconstructed_batch_numpy_squeeze = reconstructed_batch_numpy.squeeze()

from PIL import Image
for i in range(100):
    im = Image.fromarray(255*reconstructed_batch_numpy_squeeze[i]).convert("L")
    im.save(fp =  "eval/images/reconstructed/reconstructed_"+str(i).zfill(2)+".png", format = 'PNG')

ModuleAttributeError: 'ConvNet' object has no attribute 'conv6'

In [1]:
import torch
NN = torch.load('local_testing_net3')
NN

ConvNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
  (poolavg): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (poolmax): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (conv5): Conv2d(64, 1, kernel_size=(1, 1), stride=(1, 1))
)

In [3]:
import torch
NN.load_state_dict(torch.load('local_testing_net2'))

ModuleAttributeError: 'ConvNet' object has no attribute 'copy'

In [5]:
import torch
from ClassFiles.DataLoader import get_generated_dataloader
from ClassFiles.networks import SebastianConvNet
from algorithm2_old import reconstruct, shitty_reconstruct, minimum

eval_groundtruth_loader = get_generated_dataloader('eval', 'clean', batch_size = 100, shuffle = False)
eval_chanvese_loader = get_generated_dataloader('eval', 'chan-vese', batch_size = 100, shuffle = False)
eval_noisy_loader = get_generated_dataloader('eval', 'dirty', batch_size = 100, shuffle = False)

eval_groundtruth_batch = iter(eval_groundtruth_loader).next()[0].to("cuda")
eval_chanvese_batch = iter(eval_chanvese_loader).next()[0].to("cuda")
eval_chanvese_batch = (eval_chanvese_batch > 0.5).float()
eval_noisy_batch = iter(eval_noisy_loader).next()[0].to("cuda")

batch_numpy = eval_chanvese_batch.to("cpu").numpy()
batch_numpy_squeeze = batch_numpy.squeeze()

from PIL import Image
for i in range(100):
    im = Image.fromarray(255*batch_numpy_squeeze[i]).convert("L")
    im.save(fp =  "eval/images/chan-vese_binary/chan-vese_binary_"+str(i).zfill(2)+".png", format = 'PNG')

In [6]:
reconstructed_batch_numpy = reconstructed_batch.to("cpu").numpy()
reconstructed_batch_numpy_squeeze = reconstructed_batch_numpy.squeeze()

from PIL import Image
for i in range(100):
    im = Image.fromarray(255*reconstructed_batch_numpy_squeeze[i]).convert("L")
    im.save(fp =  "eval/images/reconstructed/reconstructed_"+str(i).zfill(2)+".png", format = 'PNG')

In [8]:
NN.conv1.weight.sum()

tensor(nan, device='cuda:0', grad_fn=<SumBackward0>)

In [5]:
import torch
import torch.nn.functional as F
from ClassFiles.DataLoader import get_generated_dataloader

eval_groundtruth_loader = get_generated_dataloader('eval', 'clean', batch_size = 100, shuffle = False)
eval_chanvese_loader = get_generated_dataloader('eval', 'chan-vese', batch_size = 100, shuffle = False)
eval_noisy_loader = get_generated_dataloader('eval', 'dirty', batch_size = 100, shuffle = False)

eval_groundtruth_batch = iter(eval_groundtruth_loader).next()[0].to("cuda")
eval_chanvese_batch = iter(eval_chanvese_loader).next()[0].to("cuda")
eval_noisy_batch = iter(eval_noisy_loader).next()[0].to("cuda")

(F.relu(NN(eval_groundtruth_batch) - NN(eval_chanvese_batch))).sum()

tensor(0., device='cuda:0', grad_fn=<SumBackward0>)

In [8]:
a = torch.rand([5,5,5])
b = a.view(-1).max(0)[0]
a, b

(tensor([[[0.4866, 0.4319, 0.0966, 0.3181, 0.3196],
          [0.5025, 0.4174, 0.2378, 0.6035, 0.6831],
          [0.6580, 0.7879, 0.3428, 0.0460, 0.4932],
          [0.4412, 0.9941, 0.6293, 0.0761, 0.3081],
          [0.4332, 0.1497, 0.5951, 0.3191, 0.8414]],
 
         [[0.4712, 0.1713, 0.7046, 0.1318, 0.3954],
          [0.9703, 0.0051, 0.3613, 0.8463, 0.2186],
          [0.0690, 0.0189, 0.9013, 0.0287, 0.7326],
          [0.1387, 0.3482, 0.0201, 0.8070, 0.8237],
          [0.0934, 0.3351, 0.7166, 0.0012, 0.6698]],
 
         [[0.7275, 0.3554, 0.1366, 0.4507, 0.7635],
          [0.2860, 0.0305, 0.1021, 0.1570, 0.3808],
          [0.4574, 0.4723, 0.8620, 0.9717, 0.7264],
          [0.8908, 0.9002, 0.1904, 0.2882, 0.6909],
          [0.7522, 0.2087, 0.5201, 0.4837, 0.0691]],
 
         [[0.8193, 0.4658, 0.8947, 0.0230, 0.8322],
          [0.2087, 0.4088, 0.9786, 0.2100, 0.7312],
          [0.5487, 0.5913, 0.9797, 0.7988, 0.5188],
          [0.0665, 0.4540, 0.4372, 0.5147, 0.3565],
   